<a href="https://colab.research.google.com/github/ChizuNwosu/RealTime_MachineLearning/blob/main/Homework_2_Problem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim
torch.set_printoptions(edgeitems=2, linewidth=75)
import torch.nn as nn
from collections import OrderedDict
from matplotlib import pyplot as plt

In [ ]:
import pandas as pd
csv_file = '/media/Housing.csv'
df = pd.read_csv (csv_file)   
print (df)

        price  area  bedrooms  ...  parking  prefarea furnishingstatus
0    13300000  7420         4  ...        2       yes        furnished
1    12250000  8960         4  ...        3        no        furnished
2    12250000  9960         3  ...        2       yes   semi-furnished
3    12215000  7500         4  ...        3       yes        furnished
4    11410000  7420         4  ...        2        no        furnished
..        ...   ...       ...  ...      ...       ...              ...
540   1820000  3000         2  ...        2        no      unfurnished
541   1767150  2400         3  ...        0        no   semi-furnished
542   1750000  3620         2  ...        0        no      unfurnished
543   1750000  2910         3  ...        0        no        furnished
544   1750000  3850         3  ...        0        no      unfurnished

[545 rows x 13 columns]


In [ ]:
# Converting csv file to usable tensor data
import csv
priceS = []
areaS = []
bedroomsS = []
bathroomsS = []
storiesS = []
parkingS = []

price = []
area = []
bedrooms = []
bathrooms = []
stories = []
parking = []

with open(csv_file, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        priceS.append(row.get('price'))
        areaS.append(row.get('area'))
        bedroomsS.append(row.get('bedrooms'))
        bathroomsS.append(row.get('bathrooms'))
        storiesS.append(row.get('stories'))
        parkingS.append(row.get('parking'))

price = [int(numeric_string) for numeric_string in priceS]
area = [int(numeric_string) for numeric_string in areaS]
bedrooms = [int(numeric_string) for numeric_string in bathroomsS]
bathrooms = [int(numeric_string) for numeric_string in bedroomsS]
stories = [int(numeric_string) for numeric_string in storiesS]
parking =  [int(numeric_string) for numeric_string in parkingS]

t_pr = torch.tensor(price) # t_c
t_ar = torch.tensor(area)   # t_u1
t_be = torch.tensor(bedrooms)   # t_u2
t_ba = torch.tensor(bathrooms)  # t_u3
t_st = torch.tensor(stories)    # t_u4
t_pa = torch.tensor(parking)    # t_u5

# Normalization
from math import sqrt
def tensorToNormal(t_y):
  t_y = t_y.float()
  m = t_y.mean()
  v = t_y.var()
  s = sqrt(v)
  norm = (t_y - m)/ s
  return norm

In [ ]:
t_c = tensorToNormal(t_pr)
t_u1 = tensorToNormal(t_ar)
t_u2 = tensorToNormal(t_be)
t_u3 = tensorToNormal(t_ba)
t_u4 = tensorToNormal(t_st)
t_u5 = tensorToNormal(t_pa)
t_total = t_u1 + t_u2 + t_u3 + t_u4 + t_u5

In [ ]:
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u1 = torch.tensor(t_u1).unsqueeze(1) # <1>
t_u2 = torch.tensor(t_u2).unsqueeze(1) # <1>
t_u3 = torch.tensor(t_u3).unsqueeze(1) # <1>
t_u4 = torch.tensor(t_u4).unsqueeze(1) # <1>
t_u5 = torch.tensor(t_u5).unsqueeze(1) # <1>
t_total = torch.tensor(t_total).unsqueeze(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy 

In [ ]:
# Samples of five housing variables changes
n_samples = t_total.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([193, 184,  97, 323, 312,  33, 239, 414, 503, 499, 197, 507, 437,
         375,  43, 129, 172, 392, 328, 240, 357, 450,  78, 235, 434, 247,
         513,  63, 317, 166, 429, 344,  61, 451, 110, 327, 386, 105, 497,
         366, 222, 244, 406, 155, 270, 440, 346, 415,  24, 504, 182,  72,
          45, 301, 395, 381, 153, 427,  64, 326, 385, 183, 461, 298, 151,
          67, 533, 154, 486, 112,  58, 216, 520, 139, 501, 136, 211,   7,
          14, 279, 463, 304, 416,  84, 532, 292, 149, 537, 113, 272, 147,
         433, 425, 371, 514,  95, 305, 160, 106, 332, 267,  83, 372, 180,
         118, 539, 308, 191, 148, 196, 468, 458, 133,   2, 283, 512, 176,
         354, 505,  92, 207, 517, 322, 391, 189,  19, 493, 476, 320, 521,
         349,  87, 475, 377, 237, 213, 156, 143,  75, 200, 203, 404, 396,
         471,  94,  27, 412, 145,  15, 526, 157, 221, 438, 248, 127, 343,
         177,  89, 494, 454, 275, 356, 382, 352, 484,  96, 421, 474, 490,
         487,  49, 313, 194,  81,   5,

In [ ]:
t_u_train = t_total[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_total[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [ ]:
# Problem 1 part 1

neuron_count = 8

seq_model = nn.Sequential(
    nn.Linear(1, neuron_count),
    nn.Tanh(),
    nn.Linear(neuron_count,1)
)


optimizer = optim.SGD(seq_model.parameters(), lr=1e-4)

training_loop(
    n_epochs = 200, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val) 

Epoch 1, Training loss 1.0818, Validation loss 0.8917
Epoch 10, Training loss 1.0817, Validation loss 0.8915
Epoch 20, Training loss 1.0815, Validation loss 0.8913
Epoch 30, Training loss 1.0813, Validation loss 0.8911
Epoch 40, Training loss 1.0811, Validation loss 0.8909
Epoch 50, Training loss 1.0809, Validation loss 0.8907
Epoch 60, Training loss 1.0807, Validation loss 0.8905
Epoch 70, Training loss 1.0805, Validation loss 0.8903
Epoch 80, Training loss 1.0803, Validation loss 0.8901
Epoch 90, Training loss 1.0801, Validation loss 0.8900
Epoch 100, Training loss 1.0799, Validation loss 0.8898
Epoch 110, Training loss 1.0797, Validation loss 0.8896
Epoch 120, Training loss 1.0795, Validation loss 0.8894
Epoch 130, Training loss 1.0793, Validation loss 0.8892
Epoch 140, Training loss 1.0791, Validation loss 0.8890
Epoch 150, Training loss 1.0789, Validation loss 0.8888
Epoch 160, Training loss 1.0787, Validation loss 0.8886
Epoch 170, Training loss 1.0785, Validation loss 0.8884
Epo

In [ ]:
# Problem 1 part 2

neuron_count = 8

seq_model = nn.Sequential(
    nn.Linear(1, neuron_count),
    nn.Tanh(),
    nn.Linear(neuron_count, 13),
    nn.Tanh(),
    nn.Linear(13, 5),
    nn.Tanh(),
    nn.Linear(5,1)
)


optimizer = optim.SGD(seq_model.parameters(), lr=1e-4)

training_loop(
    n_epochs = 200, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val) 

Epoch 1, Training loss 1.1657, Validation loss 0.9894
Epoch 10, Training loss 1.1650, Validation loss 0.9886
Epoch 20, Training loss 1.1642, Validation loss 0.9878
Epoch 30, Training loss 1.1634, Validation loss 0.9869
Epoch 40, Training loss 1.1626, Validation loss 0.9861
Epoch 50, Training loss 1.1618, Validation loss 0.9852
Epoch 60, Training loss 1.1610, Validation loss 0.9844
Epoch 70, Training loss 1.1603, Validation loss 0.9836
Epoch 80, Training loss 1.1595, Validation loss 0.9827
Epoch 90, Training loss 1.1588, Validation loss 0.9819
Epoch 100, Training loss 1.1580, Validation loss 0.9811
Epoch 110, Training loss 1.1573, Validation loss 0.9803
Epoch 120, Training loss 1.1565, Validation loss 0.9795
Epoch 130, Training loss 1.1558, Validation loss 0.9788
Epoch 140, Training loss 1.1551, Validation loss 0.9780
Epoch 150, Training loss 1.1544, Validation loss 0.9772
Epoch 160, Training loss 1.1537, Validation loss 0.9765
Epoch 170, Training loss 1.1530, Validation loss 0.9757
Epo